# **Real or Not? NLP with Disaster Tweets** 

Predict which Tweets are about real disasters and which ones are not.

# **ACCESO AL PROYECTO DESDE DRIVE**

In [ ]:
# Usamos google mount para tener acceso al proyecto desde drive
from google.colab import drive
drive.mount('/content/drive/')

# !ls "/content/drive/My Drive/Factored project/"  # muestra lista con todos los archivos en la dirección

# **IMPORTACIÓN LIBRERÍAS Y REVISIÓN DATOS**


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import spacy # spaCy is a leading NLP library when working with text.
nlp = spacy.load("en_core_web_sm") # Loading English Language Model
from spacy import lemmatizer

#Guardar modelos
from sklearn.externals import joblib

In [ ]:
!pip install tensorflow==2.0

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub 

In [ ]:
# Importación librerías para preprocesamiento 
import re  #Regular expression operations = RE

# STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#PUNCTUATION
import string

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Factored project/train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

In [ ]:
data = data.drop(columns= ['id', 'keyword', 'location'], axis=1)
data.head()

# **NUEVO DATAFRAME**

CREACIÓN DE NUEVO DATAFRAME PARA NO MANIPULAR EL ORIGINAL

***COLUMNAS:***


*   **text**: Tiene el texto tal cual el dataset
*   **target**: target del dataset 
*   **NoRuido**: Todo el ruido eliminado de la columna **text** (URL's, HTML tags, Emojis)
*   **PRE**: Preprocesamiento a **text**
*   **URLS**: Sólo se elimina las URL's de **text**
*   **HTML**: Sólo se elimina las HTML tags de **text**
*   **EMOJIS**: Sólo se elimina los emojis de **text**

In [ ]:
#NUEVO DATAFRAME
newdata = pd.DataFrame(columns=('text', 'target','NoRuido','PRE','URLS','HTML','EMOJIS'))
pd.set_option('display.max_colwidth',-1)
newdata['text']=data['text']
newdata['target']=data['target']

# **PREPROCESAMIENTO**

**CONTRACTIONS**

In [ ]:
#https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert


def clean(tweet): 

    tweet=tweet.lower()
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
    return tweet

In [ ]:
newdata['PRE'] = newdata['text'].apply(lambda s : clean(s))

#Se debe hacer también al test
#df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean(s))

**SLANGS**

In [ ]:
#https://www.kaggle.com/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

In [ ]:
newdata['PRE'] = newdata['PRE'].apply(lambda s : convert_abbrev(s))

**REMOVING STOPWORDS**

Sí era necesario hacer lo anterior según la lista de stopwords que se puede observar en el siguiente link
https://gist.github.com/sebleier/554280

In [ ]:
STOPWORDS = set(stopwords.words('english'))

In [ ]:
# FUENTE: https://www.kaggle.com/kamalpangeni/disaster-tweets-with-lr-rf-dnn
#Se necesita poner el texto en minúscula para poder eliminar las STOPWORDS 

def text_preprocessing(text):
    text= text.lower() 
    text = " ".join([word for word in str(text).split() if word not in STOPWORDS]) 
    return text
    

In [ ]:
newdata['PRE'] = newdata['PRE'].apply(lambda s : text_preprocessing(s))

**MULTIPLE LETTERS**

multiple  letters repeats like in 'Oooooohhh' are truncated to 2 letters, not possible to truncate to 1 letter, because it may generated false meaning like  'good' to 'god'


In [ ]:
# FUENTE: https://www.kaggle.com/dmitri9149/transformer-svm-semantically-identical-tweets

def multiple_letters(text):
    text = re.sub(r'([a-zA-Z])\1{1,}', r'\1\1', text)
    return text

In [ ]:
newdata['PRE'] = newdata['PRE'].apply(lambda s : multiple_letters(s))

**REMOVING PUNCTUATION**

In [ ]:
# Removal of punctuations 

PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
  """custom function to remove the punctuation"""
  return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


In [ ]:
newdata['PRE'] = newdata['PRE'].apply(lambda s : remove_punctuation(s))
#FALTA HACERLE AL TEST TAMBIÉN

# **ELIMINACIÓN DEL RUIDO DE TEXTO**

**ELIMINAR URL'S DE LOS TWEETS**


In [ ]:
# FUNCIÓN PARA ELIMINAR URL'S DE LOS TWEETS
# Fuente:  https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [ ]:
newdata['URLS']=newdata['text'].apply(lambda x : remove_URL(x))  

**REMOVING HTML TAGS**

In [ ]:
# Fuente:  https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
newdata['HTML']=newdata['text'].apply(lambda x : remove_html(x)) 

**REMOVING EMOJIS**

In [ ]:
# Fuente:  https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove
# Fuente: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

def remove_emoji(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

In [ ]:
newdata['EMOJIS']=newdata['text'].apply(lambda x: remove_emoji(x))

**REMOVING ALL**

In [ ]:
newdata['NoRuido']=newdata['text'].apply(lambda x : remove_URL(x)) 
newdata['NoRuido']=newdata['NoRuido'].apply(lambda x : remove_html(x)) 
newdata['NoRuido']=newdata['NoRuido'].apply(lambda x: remove_emoji(x))

In [ ]:
newdata.head()


# **MODELOS**

In [ ]:
# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split #Partición de los datos
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, f1_score # Métricas

In [ ]:
# modelos
model_1 = LogisticRegression(max_iter=500)
model_2 = SVC()
model_3 = RandomForestClassifier(n_estimators=100,n_jobs=2) #n_job=# de núcleos trabajando al mismo tiempo, la ram de CoLab tiene 2

#**MODELOS CON TEXTO VIRGEN**

En está parte analizamos los modelos de regresión logística, máquinas de soporte vectorial (SVM) y árboles de decisión (RandomForest Classifier), todos estos, ya que, nuestro problema consiste en determinar si un tweet corresponde a un desastre o no (problema de clasificación). Para los modelos emplearemos tres formas de hacer **embedding** los cuales son:



1.   Bag of words
2.   TF-IDF
3.   Tensorflow hub




## 1.**Bag Of Words**



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # BOW

v_1 = CountVectorizer()
X_B_1 = v_1.fit_transform(data['text']) # Crea la matriz de frecuencias y selecciona las características por defecto crea una matriz sparse
X_B_1 = X_B_1.toarray()

print(50*'-')
print('El número de características es: {}'.format(X_B_1.shape[1]))
print('El número de datos es: {}'.format(X_B_1.shape[0]))
print(50*'-')

In [ ]:
# Partición de los datos 
x_e_B1, x_p_B1, y_e_B1, y_p_B1 = train_test_split(X_B_1, data['target'], test_size=0.2, shuffle=True)
# x_e_B1 = X del entrenamiento
# x_p_B1 = X de prueba
# y_e_B1 = Y de entrenamiento
# y_p_B1 = Y de prueba

################ Lista para almacenar AUC en orde RL, SVM, RFC ######
L_tvirgen_BOW = []

**Regresión Logística**

In [ ]:
model_1.fit(x_e_B1, y_e_B1)
model_1_tvirgen_bow_1 = '/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_bow_1.pkl'
joblib.dump(model_1, model_1_tvirgen_bow_1)

In [ ]:
model_1_tvirgen_bow_1 = joblib.load('/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_bow_1.pkl')
y_score = model_1_tvirgen_bow_1.predict_proba(x_p_B1)[:,1]
auc_RL_tvirgen_bow = roc_auc_score(y_p_B1, y_score)
L_tvirgen_BOW.append(auc_RL_tvirgen_bow)
fpr, tpr, threshold = roc_curve(y_p_B1, y_score)
plt.grid()
plt.axis([0, 1, 0, 1])
plt.plot([0, 1], [0, 1], '--k')
plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, facecolor='grey', alpha= 0.2, label='AUC={}'.format(auc_RL_tvirgen_bow))
plt.legend()

**Máquinas de soporte vectorial**

In [ ]:
model_2 = SVC()
model_2.fit(x_e_B1, y_e_B1)

In [ ]:
# Graficamos curva ROC 
plot_roc_curve(model_2, x_p_B1, y_p_B1)

**Árboles de decisión**

In [ ]:
#ENTRENAMIENTO DEL MODELO, NO VOLVER A CORRER
model_3.fit(x_e_B1, y_e_B1)

In [ ]:
#GUARDA MODELO
joblib.dump(model_3, "/content/drive/My Drive/Factored project/Modelos/model_3_tvirgen_bow_1.pkl")

In [ ]:
#CARGAR MODELO PARA USAR
model_3=joblib.load("/content/drive/My Drive/Factored project/Modelos/model_3_tvirgen_bow_1.pkl")

In [ ]:
# Graficamos curva ROC 
plot_roc_curve(model_3, x_p_B1, y_p_B1)



```
# Tiene formato de código
```

## 2. **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v_2 = TfidfVectorizer()
X_T_1 = v_2.fit_transform(data['text'])
X_T_1 = X_T_1.toarray()

print(50*'-')
print('El número de características es: {}'.format(X_T_1.shape[1]))
print('El número de datos es: {}'.format(X_T_1.shape[0]))
print(50*'-')

In [ ]:
# Partición de los datos
x_e_T1, x_p_T1, y_e_T1, y_p_T1 = train_test_split(X_T_1, data['target'], test_size=0.2, shuffle=True)

#### Lista de resultados #### 
L_tvirgen_Tfi = [] 

**Regresión logística**

In [ ]:
model_1.fit(x_e_T1, y_e_T1)
model_1_tvirgen_tfidf_1 = '/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_tfidf_1.pkl'
joblib.dump(model_1, model_1_tvirgen_tfidf_1)


In [ ]:
model_1_tvirgen_tfidf_1 = joblib.load('/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_tfidf_1.pkl')
y_stfi = model_1_tvirgen_tfidf_1.predict_proba(x_p_T1)[:,1]
auc_RL_tvirgen_tfidf = roc_auc_score(y_p_T1, y_stfi)
L_tvirgen_Tfi.append(auc_RL_tvirgen_tfidf)
fpr, tpr, threshold = roc_curve(y_p_T1, y_stfi)
plt.grid()
plt.axis([0, 1, 0, 1])
plt.plot([0, 1], [0, 1], '--k')
plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, facecolor='grey', alpha= 0.2, label='AUC={}'.format(auc_RL_tvirgen_tfidf))
plt.legend()

**Máquinas de soporte vectorial**

In [ ]:
# Ajustamos el modelo
model_2.fit(x_e_T1, y_e_T1)

In [ ]:
# curva ROC 
plot_roc_curve(model_2, x_p_T1, y_p_T1)

In [ ]:
# Metricas F1 y AUC
print(40*'-')
print('La metrica F1 es: {}'.format(f1_score(y_p_T1,model_2.predict(x_p_T1))))
print('AUC: {}'.format(roc_auc_score(y_p_T1,model_2.predict(x_p_T1))))
print(40*'-')

**Árboles de decisión**

In [ ]:
# model_3 = RandomForestClassifier(n_estimators=100,n_jobs=2)
model_3.fit(x_e_T1, y_e_T1)

In [ ]:
# Metrica F1
print(40*'-')
print('La metrica F1 es: {}'.format(f1_score(y_p_T1,model_3.predict(x_p_T1))))
print(40*'-')

## 3. **Tensorflow hub**

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = embed(data['text'])
X = X.numpy()

In [ ]:
# Partición de los datos
x_e_Th1, x_p_Th1, y_e_Th1, y_p_Th1 = train_test_split(X, data['target'], test_size=0.2, shuffle=True)

### Lista 
L_tvirgen_tfh =[]

**Regresión Logística**

In [ ]:
model_1.fit(x_e_Th1, y_e_Th1)
model_1_tvirgen_tfh_1 = '/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_tfh_1.pkl' 
joblib.dump(model_1, model_1_tvirgen_tfh_1)

In [ ]:
model_1_tvirgen_tfh_1 = joblib.load('/content/drive/My Drive/Factored project/Modelos/model_1_tvirgen_tfh_1.pkl')
y_stfh = model_1_tvirgen_tfh_1.predict_proba(x_p_Th1)[:,1]
auc_RL_tvirgen_tfh = roc_auc_score(y_p_Th1, y_stfh)
L_tvirgen_tfh.append(auc_RL_tvirgen_tfh)
fpr, tpr, threshold = roc_curve(y_p_Th1, y_stfh)
plt.grid()
plt.axis([0, 1, 0, 1])
plt.plot([0, 1], [0, 1], '--k')
plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, facecolor='grey', alpha= 0.2, label='AUC={}'.format(auc_RL_tvirgen_tfh))
plt.legend()

**Máquinas de Soporte Vectorial**

In [ ]:
model_2.fit(x_e_Th1, y_e_Th1)

plot_roc_curve(model_2, x_p_Th1, y_p_Th1)

In [ ]:
f_1_svm1 = f1_score(y_p_Th1, model_2.predict(x_p_Th1))
print(40*'-')
print(40*'-')
print('La metrica F1 es: {}'.format(f_1_svm1))
print('AUC: {}'.format(roc_auc_score(y_p_T1,model_2.predict(x_p_T1))))
print(40*'-')
print(40*'-')


**RandomForest**

In [ ]:
model_3.fit(x_e_Th1, y_e_Th1)

plot_roc_curve(model_3, x_p_Th1, y_p_Th1)

In [ ]:
f_1_r1 = f1_score(y_p_Th1, model_3.predict(x_p_Th1))
print(40*'-')
print(40*'-')
print('La metrica F1 es: {}'.format(f_1_r1))
print('AUC: {}'.format(roc_auc_score(y_p_T1,model_3.predict(x_p_T1))))
print(40*'-')
print(40*'-')

In [ ]:
## Resultados AUC texto virgen ## 
L_tvirgen = L_tvirgen_BOW + L_tvirgen_Tfi + L_tvirgen_tfh
print(L_tvirgen)

# **MODELOS CON TEXTO SIN RUIDO**

AQUÍ SE DEBE USAR LA COLUMNA newdata['NoRuido'] 


### **Bag Of Words**

In [ ]:
X_B_2 = v_1.fit_transform(newdata['NoRuido']) # Crea la matriz de frecuencias y selecciona las características por defecto crea una matriz sparse
X_B_2 = X_B_2.toarray()
print(50*'-')
print(50*'-')
print('El número de características es: {}'.format(X_B_2.shape[1]))
print('El número de datos es: {}'.format(X_B_2.shape[0]))
print(50*'-')
print(50*'-')

In [ ]:
# Partición de los datos 
x_e_B2, x_p_B2, y_e_B2, y_p_B2 = train_test_split(X_B_2, newdata['target'], test_size=0.2)
# x_e_B2 = X del entrenamiento
# x_p_B2 = X de prueba
# y_e_B2 = Y de entrenamiento
# y_p_B2 = Y de prueba

## Lista de resultados ### 
L_tnoruido_BOW = [] 

**Regresión Logística**

In [ ]:
model_1.fit(x_e_B2, y_e_B2)
model_1_tnoruido_bow = '/content/drive/My Drive/Factored project/Modelos/model_1_tnoruido_bow.pkl'
joblib.dump(model_1, model_1_tnoruido_bow)

In [ ]:
model_1_tnoruido_bow = joblib.load('/content/drive/My Drive/Factored project/Modelos/model_1_tnoruido_bow.pkl')
y_sbow = model_1_tnoruido_bow.predict_proba(x_p_B2)[:,1]
auc_RL_tnoruido_bow = roc_auc_score(y_p_B2, y_sbow)
L_tnoruido_BOW.append(auc_RL_tnoruido_bow)
fpr, tpr, threshold = roc_curve(y_p_B2, y_sbow)
plt.grid()
plt.axis([0, 1, 0, 1])
plt.plot([0, 1], [0, 1], '--k')
plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, facecolor='grey', alpha= 0.2, label='AUC={}'.format(auc_RL_tvirgen_tfh))
plt.legend()

**Random Forest**

In [ ]:
model_3.fit(x_e_B2, y_e_B2)

## **TF-IDF**

In [ ]:
X_T_2 = v_2.fit_transform(newdata['NoRuido'])
X_T_2 = X_T_2.toarray()
print(50*'-')
print(50*'-')
print('El número de características es: {}'.format(X_T_2.shape[1]))
print('El número de datos es: {}'.format(X_T_2.shape[0]))
print(50*'-')
print(50*'-')


In [ ]:
x_e_T2, x_p_T2, y_e_T2, y_p_T2 = train_test_split(X_T_2, newdata['target'], test_size=0.2)
## Lista de Resultados ## 
L_tnoruido_tfidf =[]


**Regresión Logística**

In [ ]:
model_1.fit(x_e_T2, y_e_T2)
model_1_tnoruido_tfidf = '/content/drive/My Drive/Factored project/Modelos/model_1_tnoruido_tfidf.pkl'
joblib.dump(model_1, model_1_tnoruido_tfidf)

In [ ]:
model_1_tnoruido_tfidf = joblib.load('/content/drive/My Drive/Factored project/Modelos/model_1_tnoruido_tfidf.pkl')
y_stfi = model_1_tnoruido_tfidf.predict_proba(x_p_T2)[:,1]
L_tnoruido_tfidf.append(roc_auc_score(y_p_T2, y_stfi))
fpr, tpr, threshold = roc_curve(y_p_T2, y_stfi)
plt.grid()
plt.axis([0, 1, 0, 1])
plt.plot([0, 1], [0, 1], '--k')
plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, facecolor='grey', alpha= 0.2, label='AUC={}'.format(roc_auc_score(y_p_T2, y_stfi)))
plt.legend()


## **Tensorflow hub**

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = embed(newdata['NoRuido'])
X = X.numpy()

# **MODELOS CON PREPROCESAMIENTO**

AQUÍ SE DEBE USAR LA COLUMNA newdata['PRE']

# **MODELOS ELIMINANDO URL's**

AQUÍ SE DEBE USAR LA COLUMNA newdata['URLS']

# **MODELOS ELIMINANDO HTML TAGS**

AQUÍ SE DEBE USAR LA COLUMNA newdata['HTML']

# **MODELOS ELIMINANDO EMOJIS**

AQUÍ SE DEBE USAR LA COLUMNA newdata['EMOJIS']

# **MÁS COSAS**



In [ ]:
# JOBLIB ES PARA GUARDAR LOS MODELOS ENTRENADOS Y NO TENERLOS QUE ENTRENAR CADA VEZ QUE CORREMOS

# Guardamos el modelo
from sklearn.externals import joblib

#save the file in the current directory
name_model = "/content/drive/My Drive/Factored project/Modelos/name_model.pkl"
joblib.dump(model_2, name_model)

In [ ]:
print(data.text[0])
type(data.text[0])

In [ ]:
for token in nlp(data.text[0]):
  # print(token, token.lemma_, token.is_stop)
  if (token.is_stop==True):
    nlp(data.text[0]).re